# Transforming

In this notebook, we will explore how to use Large Language Models for text transformation tasks such as language translation, spelling and grammar checking, tone adjustment, and format conversion.

## Setup

In [200]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [201]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

## Translation

ChatGPT is trained with sources in many languages. This gives the model the ability to do translation. Here are some examples of how to use this capability.

In [202]:
prompt = f"""
Translate the following English text to Spanish: \ 
```Hi, I would like to order a blender```
"""
response = get_completion(prompt)
print(response)

Hola, me gustaría ordenar una licuadora.


In [203]:
prompt = f"""
Tell me which language this is: 
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)

This language is French.


In [204]:
prompt = f"""
Translate the following  text to French and Spanish
and English pirate: \
```I want to order a basketball```
"""
response = get_completion(prompt)
print(response)

French: ```Je veux commander un ballon de basket```
Spanish: ```Quiero ordenar una pelota de baloncesto```
English: ```I want to order a basketball```


In [205]:
prompt = f"""
Translate the following text to Spanish in both the \
formal and informal forms: 
'Would you like to order a pillow?'
"""
response = get_completion(prompt)
print(response)

Formal: ¿Le gustaría ordenar una almohada?
Informal: ¿Te gustaría ordenar una almohada?


### Universal Translator
Imagine you are in charge of IT at a large multinational e-commerce company. Users are messaging you with IT issues in all their native languages. Your staff is from all over the world and speaks only their native languages. You need a universal translator!

In [206]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
] 

In [207]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Korean: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")

Original message (The language is French.): La performance du système est plus lente que d'habitude.
The performance of the system is slower than usual.

시스템의 성능이 평소보다 느립니다. 

Original message (The language is Spanish.): Mi monitor tiene píxeles que no se iluminan.
English: "My monitor has pixels that do not light up."

Korean: "내 모니터에는 밝아지지 않는 픽셀이 있습니다." 

Original message (The language is Italian.): Il mio mouse non funziona
English: "My mouse is not working."
Korean: "내 마우스가 작동하지 않습니다." 

Original message (The language is Polish.): Mój klawisz Ctrl jest zepsuty
English: "My Ctrl key is broken"
Korean: "내 Ctrl 키가 고장 났어요" 

Original message (The language is Chinese.): 我的屏幕在闪烁
English: My screen is flickering.
Korean: 내 화면이 깜박거립니다. 



## Tone Transformation
Writing can vary based on the intended audience. ChatGPT can produce different tones.


In [208]:
prompt = f"""
Translate the following from slang to a business letter: 
'Dude, This is Joe, check out this spec on this standing lamp.'
"""
response = get_completion(prompt)
print(response)

Dear Sir/Madam,

I hope this letter finds you well. My name is Joe, and I am writing to bring your attention to a specification document regarding a standing lamp. 

I kindly request that you take a moment to review the attached spec, as it contains important details about the standing lamp in question. 

Thank you for your time and consideration. I look forward to hearing from you soon.

Sincerely,
Joe


## Format Conversion
ChatGPT can translate between formats. The prompt should describe the input and output formats.

In [209]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_completion(prompt)
print(response)

<!DOCTYPE html>
<html>
<head>
<style>
table {
  font-family: arial, sans-serif;
  border-collapse: collapse;
  width: 100%;
}

td, th {
  border: 1px solid #dddddd;
  text-align: left;
  padding: 8px;
}

tr:nth-child(even) {
  background-color: #dddddd;
}
</style>
</head>
<body>

<h2>Restaurant Employees</h2>

<table>
  <tr>
    <th>Name</th>
    <th>Email</th>
  </tr>
  <tr>
    <td>Shyam</td>
    <td>shyamjaiswal@gmail.com</td>
  </tr>
  <tr>
    <td>Bob</td>
    <td>bob32@gmail.com</td>
  </tr>
  <tr>
    <td>Jai</td>
    <td>jai87@gmail.com</td>
  </tr>
</table>

</body>
</html>


In [210]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


## Spellcheck/Grammar check.

Here are some examples of common grammar and spelling problems and the LLM's response. 

To signal to the LLM that you want it to proofread your text, you instruct the model to 'proofread' or 'proofread and correct'.

In [ ]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

In [211]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

Got this for my daughter for her birthday because she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. However, one of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. Additionally, it's a bit small for what I paid for it. I believe there might be other options that are bigger for the same price. On the positive side, it arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.


In [214]:
from redlines import Redlines

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

Got this for my daughter for her birthday <span style='color:red;font-weight:700;text-decoration:line-through;'>cuz </span><span style='color:green;font-weight:700;'>because </span>she keeps taking mine from my <span style='color:red;font-weight:700;text-decoration:line-through;'>room.  </span><span style='color:green;font-weight:700;'>room. </span>Yes, adults also like pandas <span style='color:red;font-weight:700;text-decoration:line-through;'>too.  </span><span style='color:green;font-weight:700;'>too. </span>She takes it everywhere with her, and it's super soft and <span style='color:red;font-weight:700;text-decoration:line-through;'>cute.  One </span><span style='color:green;font-weight:700;'>cute. However, one </span>of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. <span style='color:red;font-weight:700;text-decoration:line-through;'>It's </span><span style='color:green;font-weight:700;'>Additionally, it's </span>a bit small for what I paid for <span style='color:red;font-weight:700;text-decoration:line-through;'>it though. </span><span style='color:green;font-weight:700;'>it. </span>I <span style='color:red;font-weight:700;text-decoration:line-through;'>think </span><span style='color:green;font-weight:700;'>believe </span>there might be other options that are bigger for the same <span style='color:red;font-weight:700;text-decoration:line-through;'>price.  It </span><span style='color:green;font-weight:700;'>price. On the positive side, it </span>arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.

In [215]:
prompt = f"""
proofread and correct this review. Make it more compelling. 
Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

**Review of a Panda Plush Toy**

I purchased this adorable panda plush toy as a birthday gift for my daughter, who has a penchant for taking my belongings from my room. Contrary to popular belief, adults can also appreciate the charm of pandas. This cuddly companion has quickly become her constant companion, accompanying her wherever she goes. Its irresistibly soft and cute appearance is simply delightful.

However, I did notice a minor flaw in the design. One of the ears is slightly lower than the other, which appears to be unintentional asymmetry. While this does not detract significantly from the overall appeal, it is worth mentioning for the sake of accuracy.

In terms of size, I must admit that I expected a slightly larger plush toy considering its price. It would be prudent to explore alternative options that offer a more substantial size for the same cost. Nevertheless, the quality and craftsmanship of this panda plush toy are commendable.

To my surprise, the package arrived a day earlier than anticipated. This unexpected early delivery allowed me to indulge in some playtime with the panda plush toy before presenting it to my daughter. The joy it brought me during those moments only heightened my excitement to see her reaction.

In conclusion, despite its minor imperfections and relatively smaller size, this panda plush toy has captured the hearts of both my daughter and myself. Its undeniable charm and exceptional softness make it a worthwhile addition to any collection. For those seeking a delightful gift or a whimsical companion, this panda plush toy is sure to bring joy and comfort to individuals of all ages.

## Try it yourself!
Try changing the instructions to form your own review.

In [216]:
# Translation- English to Spanish

prompt = f"""
Translate the following English text to Spanish: \ 
```Hello, my name is Jacob Miesner and I am a Data Scientist.```
"""
response = get_completion(prompt)
print(response)

Hola, mi nombre es Jacob Miesner y soy un científico de datos.


In [217]:
# Language Classification
prompt = f"""
Tell me which language this is: 
```Hola, como estas?```

Your response should be one word.
"""
response = get_completion(prompt)
print(response)

Spanish


In [225]:
# Translation - Mulltiple Languages & style combined
prompt = f"""
Translate the following text to Chinese and Japanese
and English. Each output should be in the style of Donald Trump: \
```Exponential technological progress is here to stay.```
"""
response = get_completion(prompt)
print(response)

Chinese (Donald Trump style): "指数级的技术进步来势汹汹，它将永远存在。"

Japanese (Donald Trump style): "指数的な技術の進歩はここにあり、これからも続くんだ。"

English (Donald Trump style): "Exponential technological progress is tremendous, folks. It's here to stay, believe me."


In [227]:
# Translation - Formal and Informal

prompt = f"""
Translate the following text to Spanish in both the \
formal and informal forms: 
'Language Models have finally entered the public consciousness.'
"""
response = get_completion(prompt)
print(response)

Formal: Los Modelos de Lenguaje finalmente han entrado en la conciencia pública.
Informal: Los Modelos de Lenguaje finalmente han llegado a la mente de la gente.


In [230]:
# Translation - Multiple pieces of text
user_messages_examples = [
    "Meu computador está superaquecendo.",  # My computer is overheating
    "Моя камера не работает.",              # My camera is not working
    "Mein Internet ist langsam.",           # My internet is slow
    "音が出ない",                             # There is no sound
    "Le lecteur DVD ne lit pas les disques." # The DVD drive is not reading discs
]


for issue in user_messages_examples:
    prompt = f"Tell me what language this is in one word: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following text to English \
    and Spanish: ```{issue}```

    Adhere to the following output format:
    ```
    English: <text here>
    Spanish: <text here>
    ```
    """
    response = get_completion(prompt)
    print(response, "\n")

Original message (Portuguese): Meu computador está superaquecendo.
English: My computer is overheating.
Spanish: Mi computadora se está sobrecalentando. 

Original message (Russian): Моя камера не работает.
English: My camera is not working.
Spanish: Mi cámara no funciona. 

Original message (German): Mein Internet ist langsam.
English: My internet is slow.
Spanish: Mi internet está lento. 

Original message (Japanese): 音が出ない
English: No sound
Spanish: Sin sonido 

Original message (French): Le lecteur DVD ne lit pas les disques.
English: The DVD player does not read the discs.
Spanish: El reproductor de DVD no lee los discos. 



In [233]:
# Tone Transformation
prompt = f"""
Translate the following from slang to a business letter: 
'Bro, This new MoE model from Mistral is sicckkk. They keep killin \
the game! Give it a go when you get a chance.'
"""
response = get_completion(prompt)
print(response)

Dear Sir/Madam,

I am writing to express my enthusiasm for the new MoE model developed by Mistral. This innovative product has truly impressed me, and I believe it has the potential to greatly benefit our business. I highly recommend giving it a try at your earliest convenience.

Thank you for your attention to this matter.

Sincerely,
[Your Name]


In [234]:
# Format Conversion
data_json = {
  "library_books": [
    {
      "title": "The Great Gatsby",
      "author": "F. Scott Fitzgerald",
      "isbn": "978-0743273565"
    },
    {
      "title": "1984",
      "author": "George Orwell",
      "isbn": "978-0451524935"
    },
    {
      "title": "To Kill a Mockingbird",
      "author": "Harper Lee",
      "isbn": "978-0061120084"
    }
  ]
}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_completion(prompt)
print(response)

# Display in HTML format
display(HTML(response))

<!DOCTYPE html>
<html>
<head>
<style>
table {
  font-family: arial, sans-serif;
  border-collapse: collapse;
  width: 100%;
}

td, th {
  border: 1px solid #dddddd;
  text-align: left;
  padding: 8px;
}

tr:nth-child(even) {
  background-color: #dddddd;
}
</style>
</head>
<body>

<h2>Library Books</h2>

<table>
  <tr>
    <th>Title</th>
    <th>Author</th>
    <th>ISBN</th>
  </tr>
  <tr>
    <td>The Great Gatsby</td>
    <td>F. Scott Fitzgerald</td>
    <td>978-0743273565</td>
  </tr>
  <tr>
    <td>1984</td>
    <td>George Orwell</td>
    <td>978-0451524935</td>
  </tr>
  <tr>
    <td>To Kill a Mockingbird</td>
    <td>Harper Lee</td>
    <td>978-0061120084</td>
  </tr>
</table>

</body>
</html>


Title,Author,ISBN
The Great Gatsby,F. Scott Fitzgerald,978-0743273565
1984,George Orwell,978-0451524935
To Kill a Mockingbird,Harper Lee,978-0061120084


In [241]:
# Spellcheck
sentence_examples = [
    "The boy with the red hat is playing soccer.",  # ok
    "Anna her diary in her drawer.",          # missing word
    "Their going to their house with their dog.", # Homonyms
    "You're favorite book is over there on your shelf.", # Homonyms
    "It's important to know its value.",           # ok
    "The dessert was too sweet, making it too much to eat.", #ok
]


for t in sentence_examples:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "Spell check completed, nothing to fix.". Don't use 
    any punctuation around the text:

    ```{t}```

    """
    response = get_completion(prompt)
    print(response)

Spell check completed, nothing to fix.
Anna put her diary in her drawer.
Their going to their house with their dog.
Your favorite book is over there on your shelf.
Spell check completed, nothing to fix.
Spell check completed, nothing to fix.


In [243]:
# Grammar Check

text = f"""
Got this skatebord for my son for his gradution because \
he's always been talking about skating. I don't know much \
about skatebords, but he seems to love it. He uses it \
almost every day to go to school or just around the block. \
The color is bright and stands out, which I guess is cool \
for kids these days. The wheels, though, they seem a bit \
wobbely, and I'm not sure if that's how they supposed to be. \
Also, it was more expensiver than I thought it would be for \
a skatebord. There could be cheaper ones out there that are \
just as good. It came two days late, which was a bummer because \
I wanted to give it on the day of his gradution.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

# Display changes
diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

Got this skateboard for my son for his graduation because he's always been talking about skating. I don't know much about skateboards, but he seems to love it. He uses it almost every day to go to school or just around the block. The color is bright and stands out, which I guess is cool for kids these days. The wheels, though, they seem a bit wobbly, and I'm not sure if that's how they're supposed to be. Also, it was more expensive than I thought it would be for a skateboard. There could be cheaper ones out there that are just as good. It came two days late, which was a bummer because I wanted to give it on the day of his graduation.


Got this <span style='color:red;font-weight:700;text-decoration:line-through;'>skatebord </span><span style='color:green;font-weight:700;'>skateboard </span>for my son for his <span style='color:red;font-weight:700;text-decoration:line-through;'>gradution </span><span style='color:green;font-weight:700;'>graduation </span>because he's always been talking about skating. I don't know much about <span style='color:red;font-weight:700;text-decoration:line-through;'>skatebords, </span><span style='color:green;font-weight:700;'>skateboards, </span>but he seems to love it. He uses it almost every day to go to school or just around the block. The color is bright and stands out, which I guess is cool for kids these days. The wheels, though, they seem a bit <span style='color:red;font-weight:700;text-decoration:line-through;'>wobbely, </span><span style='color:green;font-weight:700;'>wobbly, </span>and I'm not sure if that's how <span style='color:red;font-weight:700;text-decoration:line-through;'>they </span><span style='color:green;font-weight:700;'>they're </span>supposed to be. Also, it was more <span style='color:red;font-weight:700;text-decoration:line-through;'>expensiver </span><span style='color:green;font-weight:700;'>expensive </span>than I thought it would be for a <span style='color:red;font-weight:700;text-decoration:line-through;'>skatebord. </span><span style='color:green;font-weight:700;'>skateboard. </span>There could be cheaper ones out there that are just as good. It came two days late, which was a bummer because I wanted to give it on the day of his <span style='color:red;font-weight:700;text-decoration:line-through;'>gradution.</span><span style='color:green;font-weight:700;'>graduation.</span>

In [246]:
# Improving text

prompt = f"""
Proofread and correct this review. Make it more dramatic.
Replicate a writing style that would garner social media engagement.
Include emojis for empahsis on certain points.
Output in markdown format.
Text: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

🌟 **Review: Best Skateboard Ever! 🛹**

OMG, I got this *amazing* skateboard for my son's graduation and let me tell you, it's been a total game-changer! 🎉 He's always been obsessed with skating, so I knew I had to get him something special. And boy, did I hit the jackpot with this one! 💥

First off, let me just say that I'm not a skateboard expert, but my son is head over heels in love with this board! He literally uses it *every single day* to cruise around the neighborhood or even go to school. 🏫 It's become his ultimate mode of transportation, and I couldn't be happier! 😍

The color of this board is *so vibrant* and eye-catching, it's like a neon dream come true! 🌈 It totally stands out from the crowd, which is super cool for all the trendy kids out there. 🤙

Now, I have to be honest here, there's just one tiny little issue... the wheels are a bit wobbly. 😬 I'm not sure if that's how they're supposed to be, but it's something to keep in mind. However, despite this minor setback, my son still manages to shred like a pro! 🤘

Okay, let's talk about the price. 💸 I have to admit, it was a bit more expensive than I initially anticipated. But hey, quality comes at a cost, right? And let me tell you, this skateboard is worth every penny! 💯 There might be cheaper options out there, but trust me, they won't even come close to the awesomeness of this one! 🙌

The only downside was that it arrived two days late, which was a total bummer because I was dying to surprise my son on his graduation day. 😔 But hey, better late than never, right? He was still over the moon when he finally got his hands on it! 🌙

In conclusion, if you're looking for a skateboard that will make your kid's dreams come true, look no further! This skateboard is the real deal, despite the wobbly wheels and the slightly higher price tag. 🌟 Your little skater will be forever grateful, and you'll be the coolest parent on the block! 🤩✨